In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

In [2]:
# Import our input dataset
read_file = pd.read_excel ("C:/Users/Admin/group9/Resources/RealEstData05132023.xlsx")
read_file.to_csv ("C:/Users/Admin/group9/Resources/RealEstData05132023.csv", 
                  index=None,
                  header=True)

wake_df = pd.DataFrame(pd.read_csv("C:/Users/Admin/group9/Resources/RealEstData05132023.csv"))
wake_df.head(10)

C:\Users\Admin\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (12,47,70,71,72,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,OWNER1,OWNER2,Mailing_address1,Mailing_Address2,Mailing_Address3,REAL_ESTATE_ID,CARD_NUMBER,NUMBER_OF_CARDS,Street_Number,Street_Prefix,...,Land_Deferred_code,Land_Deferred_Amount,Historic_Deferred_code,Historic_Deferred_Amount,RECYCLED_UNITS,Disq_and_Qual_flag,Land_Disq_and_Qual_flag,TYPE_AND_USE,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
0,NORTHGATE PROPERTIES LLC,NaN,8465 LENTIC CT,RALEIGH NC 27615-4964,NaN,1,1,1,1506,NaN,...,NaN,0,NaN,0,0,NaN,NaN,NaN,RALEIGH,27604.0
1,BONES 13 LLC,NaN,306 MIDENHALL WAY,CARY NC 27513-5569,NaN,3,1,2,6012,NaN,...,NaN,0,NaN,0,0,A,NaN,20.0,RALEIGH,27617.0
2,BONES 13 LLC,NaN,306 MIDENHALL WAY,CARY NC 27513-5569,NaN,3,2,2,6012,NaN,...,NaN,0,NaN,0,0,A,NaN,34.0,RALEIGH,27617.0
3,HARRISPARK PROPERTIES INC,NaN,608 GASTON ST STE 200,RALEIGH NC 27603-1258,NaN,4,1,1,1601,NaN,...,NaN,0,NaN,0,0,T,NaN,85.0,RALEIGH,27604.0
4,"MOGHADASS, INC",NaN,1532 PROVIDENCE RD S STE 230,WAXHAW NC 28173-8313,NaN,5,1,1,1831,NaN,...,NaN,0,NaN,0,0,A,NaN,200.0,RALEIGH,27604.0
5,RALEIGH CITY OF,NaN,PO BOX 590,RALEIGH NC 27602-0590,NaN,16,1,1,609,NaN,...,NaN,0,NaN,0,0,NaN,NaN,NaN,RALEIGH,27610.0
6,SKS PROPERTIES,NaN,PO BOX 19808,RALEIGH NC 27619-9808,NaN,17,1,1,103,NaN,...,NaN,0,NaN,0,0,NaN,NaN,NaN,RALEIGH,27607.0
7,N C BEER WHOLESALERS ASSOC,NaN,210 N PERSON ST,RALEIGH NC 27601-1047,NaN,18,1,1,210,N,...,NaN,0,NaN,0,0,A,NaN,35.0,RALEIGH,27601.0
8,"MICKENS, ROBERT EARL & ANNIE M",NaN,2457 BERTIE DR,RALEIGH NC 27610-1730,NaN,19,1,1,2457,NaN,...,NaN,0,NaN,0,1,A,NaN,1.0,RALEIGH,27610.0
9,"COREY, JAMES CURTIS & REGINA B",NaN,2848 PROVIDENCE RD,RALEIGH NC 27610-5210,NaN,20,1,1,2848,NaN,...,NaN,0,NaN,0,1,A,NaN,1.0,RALEIGH,27610.0


In [3]:
# Export just the head(10) of our data as a reference on our ReadMe.md
wake_head_df = wake_df.head(10)
wake_head_df.to_csv(r'C:/Users/Admin/group9/Resources/wake_head_df.csv', index = False, header=True)

In [4]:
# Seeing how many null values we have
wake_df.isnull().sum()

OWNER1                          0
OWNER2                     359944
Mailing_address1                0
Mailing_Address2               14
Mailing_Address3           414218
                            ...  
Disq_and_Qual_flag          82832
Land_Disq_and_Qual_flag    281472
TYPE_AND_USE                54016
PHYSICAL_CITY                2431
PHYSICAL_ZIP_CODE            4290
Length: 87, dtype: int64

In [5]:
# Drop all columns that aren't relevant
cleaner_wake_df = wake_df[['REAL_ESTATE_ID', 'Township','Deeded_Acreage', 'Total_sale_Price', 'Total_Sale_Date' 
                           ,'Assessed_Building_Value', 'Assessed_Land_Value', 'Year_Built', 'NUM_of_Rooms','HEATED_AREA', 
                           'BATH','Story_Height','TYPE_AND_USE', 'PHYSICAL_CITY','PHYSICAL_ZIP_CODE']]
cleaner_wake_df.head()
# Export just cleaner_wake_df
cleaner_wake_df.to_csv(r'C:/Users/Admin/group9/Resources/cleaner_wake_df.csv', index = False, header=True)

In [8]:
# Checking datatypes
cleaner_wake_df.dtypes

REAL_ESTATE_ID               int64
Township                     int64
Deeded_Acreage             float64
Total_sale_Price             int64
Total_Sale_Date             object
Assessed_Building_Value      int64
Assessed_Land_Value          int64
Year_Built                   int64
NUM_of_Rooms                 int64
HEATED_AREA                  int64
BATH                        object
Story_Height                object
TYPE_AND_USE               float64
PHYSICAL_CITY               object
PHYSICAL_ZIP_CODE          float64
dtype: object

In [24]:
# Checking dataset size
cleaner_wake_df.shape

(346327, 15)

In [27]:
# Seeing if we will have enough data using only single Family residential type
cleaner_wake_df.loc[cleaner_wake_df['TYPE_AND_USE'] != 1]
cleaner_wake_df.head(10)

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Total_Sale_Date,Assessed_Building_Value,Assessed_Land_Value,Year_Built,NUM_of_Rooms,HEATED_AREA,BATH,Story_Height,TYPE_AND_USE,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
8,19,1,0.21,34500,1974-01-01,134321,115000,1964,0,1828,C,A,1.0,RALEIGH,27610.0
9,20,16,0.46,35500,1983-05-18,114933,45000,1970,0,1240,A,A,1.0,RALEIGH,27610.0
10,21,1,0.43,0,NaN,132624,90000,1999,0,1037,C,A,1.0,RALEIGH,27606.0
11,22,10,0.46,37500,2004-09-16,118723,32000,1900,0,2261,C,A,1.0,WENDELL,27591.0
14,25,8,0.96,70000,1971-01-01,358290,140000,1971,0,3770,J,A,1.0,RALEIGH,27613.0
15,26,16,0.51,0,NaN,85801,55000,1956,0,996,A,A,1.0,RALEIGH,27603.0
16,30,1,0.47,380000,2015-08-12,161077,303875,1946,0,1789,C,A,1.0,RALEIGH,27607.0
17,31,13,0.39,337500,2002-07-17,273621,200000,1970,0,2463,C,A,1.0,RALEIGH,27615.0
19,33,4,0.50,32500,1979-02-06,185581,138000,1969,0,2389,E,A,1.0,RALEIGH,27607.0
20,34,1,0.70,19500,1968-01-01,126849,79200,1968,0,1280,C,A,1.0,RALEIGH,27606.0


In [34]:
# Confirming that we only have TYPE_AND_USE as 1
cleaner_wake_df['TYPE_AND_USE'].unique()

cleaner_wake_df['TYPE_AND_USE'].nunique()

1

In [31]:
# Checking datatypes
cleaner_wake_df.dtypes

REAL_ESTATE_ID               int64
Township                     int64
Deeded_Acreage             float64
Total_sale_Price             int64
Total_Sale_Date             object
Assessed_Building_Value      int64
Assessed_Land_Value          int64
Year_Built                   int64
NUM_of_Rooms                 int64
HEATED_AREA                  int64
BATH                        object
Story_Height                object
TYPE_AND_USE               float64
PHYSICAL_CITY               object
PHYSICAL_ZIP_CODE          float64
dtype: object

In [32]:
# Checking dataset shape
cleaner_wake_df.shape

(346327, 15)

In [29]:
# Seeing how many null values we have
cleaner_wake_df.isnull().sum()

REAL_ESTATE_ID                 0
Township                       0
Deeded_Acreage                 0
Total_sale_Price               0
Total_Sale_Date            21608
Assessed_Building_Value        0
Assessed_Land_Value            0
Year_Built                     0
NUM_of_Rooms                   0
HEATED_AREA                    0
BATH                        2100
Story_Height                2075
TYPE_AND_USE                   0
PHYSICAL_CITY                209
PHYSICAL_ZIP_CODE            437
dtype: int64

In [37]:
# Dropping null values
cleaned_wake_df = cleaner_wake_df.dropna()
cleaned_wake_df.head(10)

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Total_Sale_Date,Assessed_Building_Value,Assessed_Land_Value,Year_Built,NUM_of_Rooms,HEATED_AREA,BATH,Story_Height,TYPE_AND_USE,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
8,19,1,0.21,34500,1974-01-01,134321,115000,1964,0,1828,C,A,1.0,RALEIGH,27610.0
9,20,16,0.46,35500,1983-05-18,114933,45000,1970,0,1240,A,A,1.0,RALEIGH,27610.0
11,22,10,0.46,37500,2004-09-16,118723,32000,1900,0,2261,C,A,1.0,WENDELL,27591.0
14,25,8,0.96,70000,1971-01-01,358290,140000,1971,0,3770,J,A,1.0,RALEIGH,27613.0
16,30,1,0.47,380000,2015-08-12,161077,303875,1946,0,1789,C,A,1.0,RALEIGH,27607.0
17,31,13,0.39,337500,2002-07-17,273621,200000,1970,0,2463,C,A,1.0,RALEIGH,27615.0
19,33,4,0.50,32500,1979-02-06,185581,138000,1969,0,2389,E,A,1.0,RALEIGH,27607.0
20,34,1,0.70,19500,1968-01-01,126849,79200,1968,0,1280,C,A,1.0,RALEIGH,27606.0
21,35,1,0.49,319000,1996-01-16,503301,469300,1938,0,4650,F,B,1.0,RALEIGH,27608.0
24,37,1,0.13,425000,2008-04-15,203178,300000,1921,0,1890,C,C,1.0,RALEIGH,27604.0


In [38]:
# Seeing how many null values we have
cleaned_wake_df.isnull().sum()

REAL_ESTATE_ID             0
Township                   0
Deeded_Acreage             0
Total_sale_Price           0
Total_Sale_Date            0
Assessed_Building_Value    0
Assessed_Land_Value        0
Year_Built                 0
NUM_of_Rooms               0
HEATED_AREA                0
BATH                       0
Story_Height               0
TYPE_AND_USE               0
PHYSICAL_CITY              0
PHYSICAL_ZIP_CODE          0
dtype: int64

In [40]:
# Change float64 values where we don't need decimals to int64
cleaned_wake_df['PHYSICAL_ZIP_CODE'] = cleaned_wake_df['PHYSICAL_ZIP_CODE'].apply(str)

C:\Users\Admin\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
# Checking datatypes
cleaned_wake_df.dtypes

REAL_ESTATE_ID               int64
Township                     int64
Deeded_Acreage             float64
Total_sale_Price             int64
Total_Sale_Date             object
Assessed_Building_Value      int64
Assessed_Land_Value          int64
Year_Built                   int64
NUM_of_Rooms                 int64
HEATED_AREA                  int64
BATH                        object
Story_Height                object
TYPE_AND_USE               float64
PHYSICAL_CITY               object
PHYSICAL_ZIP_CODE           object
dtype: object

In [42]:
# Checking dataset shape
cleaned_wake_df.shape

(323415, 15)

In [ ]:
# Dropping index
cleaned_wake_df = df.reset_index(drop=True)

In [ ]:
# Export just cleaned_wake_df
cleaned_wake_df.to_csv(r'C:/Users/Admin/group9/Resources/cleaned_wake_df.csv', index = False, header=True)